In [9]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
print(torch.cuda.is_available())  # GPU 사용 가능 여부 (True/False)
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수
print(torch.cuda.get_device_name(0))  # 첫 번째 GPU의 이름 출력 (GPU가 있을 경우)

#########################################

#########################################
def preprocess_data(df):

    # 제거할 컬럼
    drop_columns = [
        '불임 원인 - 자궁경부 문제', '불임 원인 - 정자 면역학적 요인',
        '불임 원인 - 정자 운동성', '시술 유형', 'DI 출산 횟수', '정자 출처', '부부 부 불임 원인', '여성 부 불임 원인', '불임 원인 - 정자 형태', '대리모 여부', '불임 원인 - 정자 농도',
        '착상 전 유전 진단 사용 여부', '기증 배아 사용 여부'
    ]

    df = df.drop(columns=[col for col in drop_columns if col in df.columns])

    # ✅ 피처 엔지니어링 적용
    df["배아 수 제곱"] = df["이식된 배아 수"] ** 2

    # ✅ 연령대를 중앙값으로 변환하여 숫자로 변환
    age_mapping = {
        "만18-34세": 26,
        "만35-37세": 36,
        "만38-39세": 38.5,
        "만40-42세": 41,
        "만43-44세": 43.5,
        "만45-50세": 47.5,
        "알 수 없음": -1  # "알 수 없음"을 특별한 값으로 처리
    }

    # ✅ 매핑 적용
    df["시술 당시 나이 숫자"] = df["시술 당시 나이"].map(age_mapping)

    # 배아 개수 대비 저장된 배아 수 비율 (배아 보존율)
    df["배아 보존율"] = df["저장된 배아 수"] / (df["총 생성 배아 수"] + 1)  # 0으로 나누는 것 방지
    df["이식 배아 비율"] = df["이식된 배아 수"] / (df["총 생성 배아 수"] + 1)
    df["배아 보존율 제곱"] =  df["배아 보존율"]* df["배아 보존율"]
    # ✅ 나이와 배아 수의 비율 변수 생성
    df["배아 수 대비 나이"] = df["시술 당시 나이 숫자"] / (df["이식된 배아 수"] + 1)

    # ✅ 추가 비율 Feature 생성
    df["배아 이식 경과일 대비 나이"] = df["배아 이식 경과일"] / (df["시술 당시 나이 숫자"] + 1)
    df["난자 활용 비율"] =df["혼합된 난자 수"] / (df["수집된 신선 난자 수"]+1)

    ivf_mapping = {'0회': 0, '1회': 1, '2회': 2, '3회': 3, '4회': 4, '5회': 5, '6회 이상': 6}

    # IVF 시술 횟수 컬럼을 매핑하여 정수형으로 변환
    df["IVF 시술 횟수"] = df["IVF 시술 횟수"].map(ivf_mapping)
    df["신선 난자 대비 IVF 시술 횟수 비율"] =df["IVF 시술 횟수"] / (df["수집된 신선 난자 수"]+1)
    df["난자당 평균 혼합 횟수"] =df["혼합된 난자 수"] / (df["IVF 시술 횟수"] + 1)
    df["1~2개 이식 여부"] = (df["이식된 배아 수"].between(1, 2)).astype(int)
    df["나이 26~36 여부"] = ((df["시술 당시 나이 숫자"] >= 26) & (df["시술 당시 나이 숫자"] <= 36)).astype(int)

    # 1️⃣ 이식된 배아 수 × 시술 당시 나이 숫자
    df["이식된 배아 수 × 시술 당시 나이"] = df["이식된 배아 수"] * df["시술 당시 나이 숫자"]
    # 2️⃣ 배아 이식 경과일 대비 나이 × 배아 수 대비 나이
    df["배아 이식 경과일 대비 나이 × 배아 수 대비 나이"] = df["배아 이식 경과일 대비 나이"] * df["배아 수 대비 나이"]
    # 3️⃣ 배아 수 대비 나이 / 총 생성 배아 수
    df["배아 수 대비 나이 / 총 생성 배아 수"] = df["배아 수 대비 나이"] / (df["총 생성 배아 수"] + 1)
    # 4️⃣ 이식된 배아 수 × 배아 이식 경과일 대비 나이
    df["이식된 배아 수 × 배아 이식 경과일 대비 나이"] = df["이식된 배아 수"] * df["배아 이식 경과일 대비 나이"]
    # 5️⃣ 시술 당시 나이 숫자 / 총 생성 배아 수
    df["시술 당시 나이 숫자 / 총 생성 배아 수"] = df["시술 당시 나이 숫자"] / (df["총 생성 배아 수"] + 1)
    
    # 새로운 파생 변수 생성
    df["시도_대비_이식_비율"] = (df["임신 시도 또는 마지막 임신 경과 연수"] / (df["배아 이식 경과일"] + 1))
    df["난자_활용도"] = (df["수집된 신선 난자 수"] / (df["총 생성 배아 수"] + 1))
    
    numeric_columns = [
        '임신 시도 또는 마지막 임신 경과 연수',
        "총 생성 배아 수",
        "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수",
        "이식된 배아 수",
        "미세주입 배아 이식 수",
        "저장된 배아 수",
        "미세주입 후 저장된 배아 수",
        "해동된 배아 수",
        "해동 난자 수",
        "수집된 신선 난자 수",
        "저장된 신선 난자 수",
        "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수",
        "기증자 정자와 혼합된 난자 수",
        "난자 채취 경과일",
        "난자 해동 경과일",
        "난자 혼합 경과일",
        "배아 이식 경과일",
        "배아 해동 경과일"
        ]
    
    df[numeric_columns] = df[numeric_columns].fillna(0)

    # **PCA**로 차원 축소 후 군집화 수행 (군집 수는 3~6 사이로 실험)
    pca = PCA(n_components=7, random_state=42)
    X_pca = pca.fit_transform(df[numeric_columns])
    kmeans = KMeans(n_clusters=5, random_state=42)
    kmeans_clusters = kmeans.fit_predict(X_pca)
    
    df['cluster_group_1'] = kmeans_clusters
    
    pca = PCA(n_components=5, random_state=42)
    X_pca = pca.fit_transform(df[numeric_columns])
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans_clusters = kmeans.fit_predict(X_pca)
    
    df['cluster_group_2'] = kmeans_clusters

    return df

# ✅ 데이터 로드
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

train = preprocess_data(train)
test = preprocess_data(test)

# LightGBM을 위한 범주형 변수 변환
train["시술 시기 코드"] = train["시술 시기 코드"].astype("category")
test["시술 시기 코드"] = test["시술 시기 코드"].astype("category")

# ✅ ID 제거
train = train.drop(columns=["ID"])
test_id = test["ID"]
test= test.drop(columns=["ID"])

hyperparameters = {
    # ✅ GPU 지원 모델 (속도 빠르고 성능 우수)
    "GBM": {},  # LightGBM (트리 부스팅, 속도 빠름)
    "XGB": {},  # XGBoost (트리 부스팅, 강력한 성능)
    "CAT": {},
    "NN_TORCH": {},
    "FASTAI": {},

    # ✅ CPU 지원 모델 (앙상블 다양성 확보)
   "RF": {},  # Random Forest (앙상블 성능 향상)
   "XT": {}   # Extra Trees (랜덤성이 강한 트리 모델)
}

hyperparameter_tune_kwargs = {
    'num_trials': 100,
    'searcher': 'bayes',  # 베이지안 최적화 방식
    'scheduler': 'local'
}
# ✅ 모델 저장 경로
save_path = "./model_save"

# 모델 학습
predictor = TabularPredictor(label="임신 성공 여부", problem_type="binary" ,eval_metric="roc_auc",path=save_path)
predictor.fit(train, presets="best_quality",
              time_limit=6000,
              hyperparameters=hyperparameters,
              hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,  # ✅ 하이퍼파라미터 튜닝 추가
              num_bag_folds=10,
              auto_stack="True",
              dynamic_stacking=False,  # DyStack 비활성화
              refit_full=True,
              calibrate=True,  # ✅ 확률값 보정 활성화,
              num_stack_levels=2
              )

Fitted model: NeuralNetTorch_BAG_L3/faec2fd3 ...
	0.7398	 = Validation score   (roc_auc)
	257.35s	 = Training   runtime
	4.44s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L3/81457484 ...
	0.7352	 = Validation score   (roc_auc)
	259.28s	 = Training   runtime
	11.59s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L3/7fec8eff ...
	0.7364	 = Validation score   (roc_auc)
	264.39s	 = Training   runtime
	18.68s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L3/9588497f ...
	0.7397	 = Validation score   (roc_auc)
	254.58s	 = Training   runtime
	6.91s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L3/96c24cdc ...
	0.7392	 = Validation score   (roc_auc)
	53.75s	 = Training   runtime
	8.16s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L3/e78cf8e2 ...
	0.7379	 = Validation score   (roc_auc)
	48.36s	 = Training   runtime
	6.53s	 = Validation runtime
Fitting model: WeightedEnsemble_L4 ... Training model for up to 360.00s of the 965.27s of remaining time.


In [10]:
# 예측
pred = predictor.predict_proba(test)  # ID 제거 후 예측
submission = pd.DataFrame({"ID": test_id, "probability": pred[1]})  # 1의 확률 사용

# 결과 병합 및 저장
submission.to_csv("Submission_PG.csv", index=False)

# 2️⃣ 가장 성능이 좋은 개별 모델 가져오기 (최신 AutoGluon 방식)
best_model = predictor.model_best

# 3️⃣ 보정 전 확률값 예측 (개별 모델 사용)
pred_uncalibrated = predictor.predict_proba(test, model=best_model)

# 4️⃣ 보정 전 결과 저장
submission_uncalibrated = pd.DataFrame({
    "ID": test_id,
    "probability": pred_uncalibrated[1]  # 1 클래스의 확률
})

# 5️⃣ CSV 파일로 저장
submission_uncalibrated.to_csv("improved_ansamble_autogluonV2.csv", index=False)

print("Submission file created: Submission_uncalibrate.csv")

# 리더보드 출력
predictor.leaderboard(silent=False)

# 1. Get the individual models and their weights from the ensemble
ensemble_model = predictor._trainer.load_model(best_model)

# 2. Access the model weights through the '_get_model_weights' method
model_weights = ensemble_model._get_model_weights() # Access the weights using the method

# 3. Print the model names and their corresponding weights
print("앙상블 모델 가중치:")
for model_name, weight in model_weights.items():
    print(f"  {model_name}: {weight}")

importance = predictor.feature_importance(train)
print(importance)

importance.head(40)

importance.tail(33)

# '특정 시술 유형' 별 '임신 성공 여부' 개수 확인
success_counts = train.groupby("총 생성 배아 수")["임신 성공 여부"].value_counts().unstack().fillna(0).astype(int)

# 결과 출력
print(success_counts)


These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '난자 채취 경과일']
Computing feature importance via permutation shuffling for 73 features using 5000 rows with 5 shuffle sets...


Submission file created: Submission_uncalibrate.csv
                                    model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                     WeightedEnsemble_L4   0.741325     roc_auc     495.739956  7109.872331                0.041535          56.171153            4       True        109
1                     WeightedEnsemble_L3   0.741132     roc_auc     357.514126  4702.242739                0.038728          34.242203            3       True         74
2                     WeightedEnsemble_L2   0.741012     roc_auc      57.334091  1617.968716                0.039353          36.138071            2       True         39
3                     LightGBM_BAG_L3/T14   0.740551     roc_auc     477.309170  7023.160859                1.030380          14.885583            3       True         88
4                      CatBoost_BAG_L3/T1   0.740540     roc_auc     476.825157  7156.272330 

	19617.16s	= Expected runtime (3923.43s per shuffle set)
	13200.93s	= Actual runtime (Completed 5 of 5 shuffle sets)


                 importance    stddev   p_value  n  p99_high   p99_low
시술 당시 나이 숫자        0.011616  0.003067  0.000533  5  0.017931  0.005301
배아 수 대비 나이         0.011504  0.002700  0.000339  5  0.017064  0.005945
배아 이식 경과일 대비 나이    0.010040  0.002282  0.000299  5  0.014740  0.005341
난자_활용도             0.007061  0.001125  0.000075  5  0.009377  0.004745
시술 시기 코드           0.006759  0.000939  0.000044  5  0.008692  0.004825
...                     ...       ...       ... ..       ...       ...
DI 임신 횟수           0.000152  0.000100  0.013575  5  0.000357 -0.000053
저장된 신선 난자 수        0.000134  0.000149  0.057462  5  0.000441 -0.000173
난자 혼합 경과일          0.000121  0.000180  0.103368  5  0.000491 -0.000249
배아 해동 경과일          0.000088  0.000088  0.044759  5  0.000270 -0.000094
난자 해동 경과일          0.000000  0.000000  0.500000  5  0.000000  0.000000

[73 rows x 6 columns]
임신 성공 여부       0     1
총 생성 배아 수             
0.0        49660  9980
1.0        13779  1444
2.0        15734  3456
3.0       